In [3]:
import networkx as nx
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.animation import FFMpegWriter
import random

# IMPORTANT: Update this path to point to your ffmpeg installation
matplotlib.rcParams['animation.ffmpeg_path'] = "/opt/homebrew/bin/ffmpeg"

# Global Animation Settings
FPS = 15
DURATION_SEC = 20
TOTAL_FRAMES = FPS * DURATION_SEC

# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)

In [4]:
def generate_hubs_graph(N=300):
    """Scenario A: Scale-Free Network (Hubs)"""
    G = nx.barabasi_albert_graph(N, 2, seed=42)
    pos = nx.spring_layout(G, seed=42)
    
    # Attributes: High degree nodes likely to be Status 1 (Red/Has Narcan)
    degrees = dict(G.degree())
    threshold = np.percentile(list(degrees.values()), 85)
    
    count = 0
    for n in G.nodes():
        # High Degree = 75% chance of Narcan
        # Low Degree = 25% chance of Narcan
        prob = 0.75 if G.degree[n] >= threshold else 0.25
        status = 1 if random.random() < prob else 0
        G.nodes[n]['status'] = status
        G.nodes[n]['color'] = '#d62728' if status == 1 else '#1f78b4'
        count += status
        
    return G, pos, count/N

def generate_sbm_graph(N=300, bridge_prob=0.0005):
    """Scenarios B, C, D: Stochastic Block Model"""
    # High internal connection, Low external connection
    probs = [[0.1, bridge_prob], [bridge_prob, 0.1]]
    sizes = [N//2, N//2]
    G = nx.stochastic_block_model(sizes, probs, seed=42)
    
    # Manual separation for visualization
    pos = nx.spring_layout(G, seed=42, k=0.15)
    for n in G.nodes():
        if G.nodes[n]['block'] == 0:
            pos[n][0] -= 0.6
        else:
            pos[n][0] += 0.6
            
    # Attributes: Group 0 mostly Red, Group 1 mostly Blue
    count = 0
    for n in G.nodes():
        group = G.nodes[n]['block']
        prob = 0.95 if group == 0 else 0.05
        status = 1 if random.random() < prob else 0
        G.nodes[n]['status'] = status
        G.nodes[n]['color'] = '#d62728' if status == 1 else '#1f78b4'
        count += status
        
    return G, pos, count/N

In [5]:
def run_step(G, current_node, homophily=1):
    """Moves the walker one step based on homophily settings."""
    neighbors = list(G.neighbors(current_node))
    if not neighbors: return current_node
    
    if homophily > 1:
        current_status = G.nodes[current_node]['status']
        weights = []
        for n in neighbors:
            if G.nodes[n]['status'] == current_status:
                weights.append(homophily)
            else:
                weights.append(1)
        total = sum(weights)
        probs = [w/total for w in weights]
        next_node = random.choices(neighbors, weights=probs, k=1)[0]
    else:
        next_node = random.choice(neighbors)
        
    return next_node

def calculate_vh_estimate(vals, degs):
    """
    Calculates the Volz-Heckathorn estimate.
    Equation: Sum(val/deg) / Sum(1/deg)
    """
    num = sum(v/d for v, d in zip(vals, degs))
    den = sum(1/d for d in degs)
    return num/den if den > 0 else 0

In [14]:
def render_scenario(filename, title, G, pos, true_prop, seeds, homophily=1, mode="standard"):
    """
    Modes:
    - 'standard': Single walker, shows Sample Mean.
    - 'vh_comparison': Single walker, shows Naive Estimate vs Volz-Heckathorn Estimate.
    - 'variance': 10 walkers, shows Pooled Mean vs Individuals.
    """
    print(f"Rendering {filename}...")
    
    fig = plt.figure(figsize=(12, 6), dpi=150)
    gs = fig.add_gridspec(1, 2, width_ratios=[1.5, 1])
    ax_net = fig.add_subplot(gs[0])
    ax_plot = fig.add_subplot(gs[1])
    fig.suptitle(title, fontsize=16)

    # 1. Calculate Fixed Axes
    all_x = [p[0] for p in pos.values()]
    all_y = [p[1] for p in pos.values()]
    margin = 0.1
    xlim = (min(all_x) - margin, max(all_x) + margin)
    ylim = (min(all_y) - margin, max(all_y) + margin)

    # 2. Setup Legend Elements
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Has Narcan',
               markerfacecolor='#d62728', markersize=10),
        Line2D([0], [0], marker='o', color='w', label='No Narcan',
               markerfacecolor='#1f78b4', markersize=10),
        Line2D([0], [0], marker='o', color='w', label='Walker',
               markerfacecolor='yellow', markersize=10, markeredgecolor='black')
    ]

    # Setup Writers
    metadata = dict(title=title, artist='Student Name')
    writer = FFMpegWriter(fps=FPS, metadata=metadata, bitrate=2000)
    
    # Simulation State
    walkers = [s for s in seeds]
    
    # Data Storage
    # We store separate lists for each walker to track their individual paths
    walker_vals = [[] for _ in walkers]
    walker_degs = [[] for _ in walkers] # Degrees needed for VH
    
    pooled_means = []
    vh_means = [] # Only used for 'vh_comparison'
    
    with writer.saving(fig, filename, dpi=150):
        for frame in range(TOTAL_FRAMES):
            
            # --- Logic Step ---
            current_vals = [] # For this specific step
            
            for i, w in enumerate(walkers):
                nxt = run_step(G, w, homophily)
                walkers[i] = nxt
                
                # Record Data
                val = G.nodes[nxt]['status']
                deg = G.degree[nxt]
                
                walker_vals[i].append(val)
                walker_degs[i].append(deg)
                current_vals.append(val)

            # --- Calculation Step ---
            
            # Pooled Naive Mean (All walkers combined)
            all_collected_vals = [v for sublist in walker_vals for v in sublist]
            pooled_means.append(np.mean(all_collected_vals))
            
            # VH Estimate
            if mode == 'vh_comparison':
                all_collected_degs = [d for sublist in walker_degs for d in sublist]
                vh = calculate_vh_estimate(all_collected_vals, all_collected_degs)
                vh_means.append(vh)

            # --- Visualization Step ---
            # Render every 2nd frame to save time/file size, or every frame for smoothness
            if frame % 2 == 0:
                ax_net.clear()
                ax_plot.clear()

                # A. Network Plot
                node_colors = [G.nodes[n]['color'] for n in G.nodes()]
                nx.draw(G, pos, ax=ax_net, node_color=node_colors, node_size=30, 
                        alpha=0.3, width=0.5, edge_color='#888')
                
                # Draw Walkers
                nx.draw_networkx_nodes(G, pos, ax=ax_net, nodelist=walkers, 
                                       node_color='yellow', node_size=150, edgecolors='black')
                
                ax_net.set_xlim(xlim)
                ax_net.set_ylim(ylim)
                ax_net.set_title(f"Step {frame}")
                ax_net.legend(handles=legend_elements, loc='upper left', fontsize='small')

                # B. Plot Graph
                ax_plot.set_ylim(0, 1)
                ax_plot.set_xlim(0, TOTAL_FRAMES)
                ax_plot.axhline(true_prop, color='black', lw=2, label='True Proportion')
                
                if mode == 'vh_comparison':
                    ax_plot.plot(pooled_means, color='red', lw=2, label='Naive (Biased)')
                    ax_plot.plot(vh_means, color='green', lw=2, label='Volz-Heckathorn (Corrected)')
                    ax_plot.set_title("The Friendship Paradox Bias")
                    
                elif mode == 'variance':
                    # Draw faint lines for individual walkers
                    for i in range(len(walkers)):
                        means = [np.mean(walker_vals[i][:k+1]) for k in range(len(walker_vals[i]))]
                        ax_plot.plot(means, color='gray', alpha=0.2, lw=1)
                    ax_plot.plot(pooled_means, color='blue', lw=2, label='Pooled Estimate (10 Chains)')
                    ax_plot.set_title("Variance Reduction")
                    
                else: # Standard mode
                    ax_plot.plot(pooled_means, color='purple', lw=2, label='Sample Mean')
                    ax_plot.set_title("Estimate Convergence")

                ax_plot.legend(loc='lower right', fontsize='small')
                ax_plot.grid(True, alpha=0.3)
                ax_plot.set_ylabel("Estimated Proportion")
                
                writer.grab_frame()
    
    plt.close(fig)
    print(f"Saved {filename}")

In [13]:
# 1. Generate Graphs
print("Generating graphs...")
G_hub, pos_hub, true_hub = generate_hubs_graph()
G_sbm, pos_sbm, true_sbm = generate_sbm_graph(bridge_prob=0.0003) # Extreme bottleneck for demo

# 2. Render Animation 1: Degree Bias (With VH Comparison)
seed_hub = random.choice(list(G_hub.nodes()))
render_scenario("animation1.mp4", "Part A: Degree Bias (Hubs)", 
                G_hub, pos_hub, true_hub, [seed_hub], 
                mode="vh_comparison")

# 3. Render Animation 2: Bottlenecks
seed_sbm_bad = [n for n in G_sbm.nodes if G_sbm.nodes[n]['block'] == 1][0]
render_scenario("animation2.mp4", "Part B: Bottlenecks (Getting Stuck)", 
                G_sbm, pos_sbm, true_sbm, [seed_sbm_bad], 
                mode="standard")

# 4. Render Animation 3: Homophily
render_scenario("animation3.mp4", "Part C: Homophily (Trust Barriers)", 
                G_sbm, pos_sbm, true_sbm, [seed_sbm_bad], 
                homophily=20, mode="standard")

# 5. Render Animation 4: Multi-Seed
seeds_multi = random.sample(list(G_sbm.nodes()), 10)
render_scenario("animation4.mp4", "Part D: Multi-Seed Correction", 
                G_sbm, pos_sbm, true_sbm, seeds_multi, 
                mode="variance")

Generating graphs...
Rendering animation1.mp4...
Saved animation1.mp4
Rendering animation2.mp4...
Saved animation2.mp4
Rendering animation3.mp4...
Saved animation3.mp4
Rendering animation4.mp4...
Saved animation4.mp4
